# COMS W4705 - Homework 4
## Question Answering with Retrieval Augmented Generation

Anubhav Jangra \<aj3228@columbia.edu\>, Emile Al-Billeh \<ea3048@columbia.edu\>, Daniel Bauer \<bauer@cs.columbia.edu\>

In this assignment, you will use a pretrained LLM for question answering on a subset of the Stanford QA Dataset (SQuAD). Here is an example question from SQuAD: 

> *Who is the headmaster of the Christian Brothers of Ireland Stella Maris College?*

Specific domain knowledge to answer questions like this may not be available in the data that the LLM was pre-trained on. As a result, if we simply prompt the the LLM to answer this question, it may tell us that it does not know the answer, or worse, it may hallucinate an incorrect answer. Even if we are lucky and the LLM has have enough information to answer this question from pre-training, but the information may be outdated (the headmaster is likely to change from time to time). 

Luckily, SQuAD provides a context snippet for each question that may contain the answer, such as

> *The Christian Brothers of Ireland Stella Maris College is a private, co-educational, not-for-profit Catholic school located in the wealthy residential southeastern neighbourhood of Carrasco. Established in 1955, it is regarded as one of the best high schools in the country, blending a rigorous curriculum with strong extracurricular activities. **The school's headmaster, history professor Juan Pedro Toni**, is a member of the Stella Maris Board of Governors and the school is a member of the International Baccalaureate Organization (IBO). Its long list of distinguished former pupils includes economists, engineers, architects, lawyers, politicians and even F1 champions. The school has also played an important part in the development of rugby union in Uruguay, with the creation of Old Christians Club, the school's alumni club.*

If we include the context as part of the prompt to the LLM, the model should be able to correctly answer the question (SQuAD contains "unanswerable questions", for which the provided context does not provide sufficient information to answer the question -- we will ignore these for the purpose of this assignment).

We will consider a scenario in which we don't know which context belongs to which question and we will use **Retrieval Augmented Generation (RAG)** techniques to identify the relevant context from the set of all available contexts. 

Specifically we will experiment with the following systems: 

* A baseline "vanilla QA" system in which we try to answer the question without any additional context (i.e. using the pre-trained LLM only).
* An "oracle" system, in which we provide the correct context for each question. This establishes an upper bound for the retrieval approaches. 
* Two different approaches for retrieving relevant contexts:
  * based on token overlap between the question and each context.
  * based on cosine similarity between question embeddings and candidate context embeddings (obtained using BERT).
    
We will evaluate each system using a number of metrics commonly used for QA tasks: 
* Exact Match (EM), which measures the percentage of predictions that exactly match the ground truth answers.
* F1 score, measured on the token overlap between the predicted and ground truth answers.
* ROUGE (specifically, ROUGE2)

Follow the instructions in this notebook step-by step. Much of the code is provided and just needs to be run, but some sections are marked with todo. Make sure to complete all these sections.


Requirements: 
Access to a GPU is required for this assignment. If you have a recent mac, you can try using mps. Otherwise, I recommend renting a GPU instance through a service like vast.ai or lambdalabs. Google Colab can work in a pinch, but you would have to deal with quotas and it's somewhat easy to lose unsaved work.

First, we need to ensure that transformers is installed, as well as the accelerate package.

In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate

Now all the relevant imports should succeed: 

In [ ]:
import os
import json
import tqdm
import copy
import torch
import torch.nn.functional as F

import re
import string
import collections

import transformers

## Data Preparation

This section creates the benchmark data we need to evaluate the QA systems. It has already been implemented for you. We recommend that you run it only once, save the benchmark data in a json file and then load it when needed. The following code may not work in Windows. We are providing the pre-generated benchmark data for download as an alternative. 

In [ ]:
data_dir = "./squad_data"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

### Downloading the Data and Creating the Benchmark Dataset

In [ ]:
training_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
val_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"

os.system(f"curl -L {training_url} -o {data_dir}/squad_train.json")  

In [ ]:
# load the raw dataset
train_data = json.load(open(f"{data_dir}/squad_train.json"))

# Some details about the dataset

# SQuAD is split up into questions about a number of different topics
print(f"Number of topics: {len(train_data['data'])}")

# Let's explore just one topic. Each topic comes with a number of context paragraphs. 
print("="*30)
print(f"For topic \"{train_data['data'][0]['title']}\"")
print(f"Number of available context paragraphs: {len(train_data['data'][0]['paragraphs'])}")
print("="*30)

print("The first paragraph is:")
print(train_data['data'][0]['paragraphs'][0]['context'])
print("="*30)

# Each paragraph comes with a number of question/answer pairs about the text in the paragraph
print("The first five question-answer pairs are:")
for qa in train_data['data'][0]['paragraphs'][0]['qas'][:5]:
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answers'][0]['text']}")
    print("-"*20)

In [ ]:
print("Total number of paragraphs in the training set:", sum([len(topic['paragraphs']) for topic in train_data['data']]))
print("Total number of question-answer pairs in the training set:", sum([len(paragraph['qas']) for topic in train_data['data'] for paragraph in topic['paragraphs']]))

In [ ]:
# not all questions are answerable given the information in the paragraph. Part of the original SQuaD 2 task is to identify such
# unanswerable questions. We will ignore them for the purpose of this assignment. 
print("Avg number of answers per question:", 
      sum([len(qa['answers']) for topic in train_data['data'] for paragraph in topic['paragraphs'] for qa in paragraph['qas']]) / 
      sum([len(paragraph['qas']) for topic in train_data['data'] for paragraph in topic['paragraphs']]))
print("Count of answerable vs unanswerable questions:")
answerable_count = 0
unanswerable_count = 0
for topic in train_data['data']:
    for paragraph in topic['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0:
                answerable_count += 1
            else:
                unanswerable_count += 1
print(f"Answerable questions: {answerable_count} ({answerable_count / (answerable_count + unanswerable_count) * 100:.2f}%)")
print(f"Unanswerable questions: {unanswerable_count} ({unanswerable_count / (answerable_count + unanswerable_count) * 100:.2f}%)")

In [ ]:
# Finally, create the RAG QA benchmark consisting of 250 answerable questions. 

# We will use all available context paragraphs for RAG
rag_contexts = [paragraph['context'] for topic in train_data['data'] for paragraph in topic['paragraphs']]

qa_pairs = []
for topic in train_data['data']:
    for paragraph in topic['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0:
                qa_pairs.append({
                    "question": qa['question'],
                    "answer": qa['answers'][0]['text'],
                    "context": paragraph['context']
                })
            
# randomly sample 250 answerable questions for the benchmark
import random
random.seed(42) # IMPORTANT so everyone is working on the same set of sampled QA pairs
sampled_qa_pairs = random.sample(qa_pairs, 250)


evaluation_benchmark = {'qas': sampled_qa_pairs, 
                        'contexts': rag_contexts}
random.shuffle(evaluation_benchmark['qas'])
random.shuffle(evaluation_benchmark['contexts'])

# save the evaluation benchmark to a file
json.dump(evaluation_benchmark, open(f"{data_dir}/rag_qa_benchmark.json", "w"), indent=2)

### Loading the Benchmark Dataset / Understanding the Data Format

Use the following code to load the benchmark data from a file. Take a look at the example output to see how the data is structured. 

In [ ]:
# load the benchmark and display some samples
evaluation_benchmark = json.load(open(f"{data_dir}/rag_qa_benchmark.json"))

print("Sample RAG contexts:")
for context in evaluation_benchmark['contexts'][:2]:
    print(context)
    print("-"*20)
print("="*30)
print("Sample RAG QA pairs:")
for qa in evaluation_benchmark['qas'][:5]:
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answer']}")
    print("-"*20)

The `evaluation_benchmark` is a dictionary with two keys: 
* `evaluation_benchmark['qas']`  provides a list of *qa_items* (see below).
* `evaluation_benchmark['contexts']` provides a list of available candidate contexts. Note that this includes all contexts from SQuAD, not just the ones for the 250 questions we sampled for the benchmark.

Each *qa_item* is a dictionary with the following keys: 
* `qa_item['question']` is the question string
* `qa_item['answer']` is the target answer string
* `qa_item['context']` is the gold context for this question

For example: 


In [ ]:
qa_items = evaluation_benchmark['qas']
len(qa_items)

In [ ]:
qa_item = qa_items[0]
qa_item['question']

In [ ]:
qa_item['answer']

In [ ]:
qa_item['context']

## Part 1 - Question Answering Evaluation Functions

In this section. we will define a number of evaluation functions that measure the quality of the QA output, compared to a single target answer for each question. 

Because the evaluation will happen at a token leve, we will perform some simple pre-processing: 

In [ ]:
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

First, Exact Match (EM) measures the percentage of predictions that match any one of the ground truth answers exactly after normalization.
The following function returns 1 if the predicted answer is correct and 0 otherwise. 

In [ ]:
def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

The next function calculates the $F_1$ score of the set of predicted tokens against the set of target tokens. 
$F_1$ is the harmonic mean of precision and recall, providing a balance between the two. Specifically 

$F_1 = \frac{2 \times \text{precision} \times \text{recall}}{\text{precision} + \text{recall}}$

where $\text{precision}$ is the fraction of predicted tokens that also appear in the target and $\text{recall}$ is the fraction of target tokens that also appear in the prediction. 

**TODO**: Write the function compute_f1(a_gold, a_pred) that returns the F1 score as defined above. It should work similar to the compute_exact method above. Test your function on a sample answer and prediction to verify that it works correctly. 

In [ ]:
def compute_f1(a_gold, a_pred): # Complete the function
  f1 = 0 # replace
  return f1

In [192]:
# Test your function

Finally, we are also want to compute ROUGE-2 scores (which extends the F1 score above to 2-grams). We can use the `rouge_score` package to do this for us. 

In [ ]:
!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer

rouge_scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=False)

def compute_rouge2(a_gold, a_pred):
    if not a_gold or not a_pred:
        return 0.0
    scores = rouge_scorer.score(a_gold.lower(), a_pred.lower())
    return scores['rouge2'].fmeasure

Let's test the metrics: 

In [ ]:
reference_answers = ["London", "The capital of England is London.", "London is the capital city of England."]
predicted_answers = ["London, capital of England"] * len(reference_answers)

print("Normalized Answers:")
for ref, pred in zip(reference_answers, predicted_answers):
    print(f"Original:")
    print(f"Reference: {ref} | Predicted: {pred}")
    print(f"Normalized:")
    print(f"Reference: {normalize_answer(ref)} | Predicted: {normalize_answer(pred)}")
    print("Exact Match:", compute_exact(normalize_answer(ref), normalize_answer(pred)))
    print("F1 Score:", compute_f1(normalize_answer(ref), normalize_answer(pred)))
    print("ROUGE-2 F1-score:", compute_rouge2(normalize_answer(ref), normalize_answer(pred)))
    print("-"*40)

## Part 2 - Vanilla Question Answering

In this part, we will use an off-the-shelf pretrained LLM and attempt to answer the questions from its pretraining knowledge only. 
To make things simple, we will use the huggingface transformer pipeline abstraction. The pipeline will download the model and parameters for us on creation. When we pass an input prompt to the pipeline, it will automatically perform preprocessing (tokenization), inference, and postprocessing (removing EOS markers and padding).

### Loading the LLM
The LLM we will use is the 1B version of the instruction tuned OLMo2 model. OLMo is an open source language model created by Allen AI and the University of Washington. Unlike other open source models, OLMo is also open data. You can read more about it here: https://huggingface.co/allenai/OLMo-2-0425-1B-Instruct and here https://allenai.org/olmo.

In [ ]:
qa_model = "allenai/OLMo-2-0425-1B-Instruct"

from transformers import pipeline

# Check which GPU device to use. Note, this will likely NOT work on a CPU. 
if torch.cuda.is_available():
    device = "cuda" 
elif torch.backends.mps.is_available():
    device = "mps" 
else:
    device = "cpu"

pipe = pipeline(
    "text-generation",
    model=qa_model,
    dtype=torch.bfloat16,
    device_map=device,
)

We can now pass a prompt to the model and retreive the completed answer. 

In [ ]:
prompt = "My favorite thing to do in fall is"
output = pipe(prompt, 
              max_new_tokens=128,
              do_sample=True, # set to False for greedy decoding below
              pad_token_id=pipe.tokenizer.eos_token_id)
print(output)

We can skip the prompt that is repeated in the output. 

In [ ]:
output[0]['generated_text'][len(prompt):].strip()

### Using the LLM for Question Answering

**TODO:** Write a function `vanilla_qa(qa_item)` that take a qa_item in the format described above, inserts the question (and only the question!) into a suitable prompt, passes the prompt to the LLM and then returns the answer as a string. 

A prompt might look like this, but will need a bit of prompt engineering to make it work well. 

> *Answer the following question concisely.* 
>
> *Question: Who played he lead role in Alien?*
> 
> *Answer:*

Once you have a basic version of the vanilla QA system you can tune the prompt (see below). 

In [ ]:
def vanilla_qa(qa_item): # Complete this function
    pass 

The following code should return an answer (but possibly not the right one) to the first question in the dataset.

In [ ]:
qa_item = evaluation_benchmark['qas'][0]
qa_item['question']

In [ ]:
vanilla_qa(qa_item) # inspect the item

And the following function evaluates the performance of your `vanilla_qa` function on a list of qa_items. 

In [ ]:
def evaluate_qa(qa_function, qa_items, verbose=False):
    results = []

    
    for i, qa_item in tqdm.tqdm(enumerate(qa_items), desc="Evaluating QA instances", total=len(qa_items)):

        question = qa_item['question'] 
        answer = qa_item['answer']
        context = qa_item['context']
        
        predicted_answer = qa_function(qa_item)

        exact_match = compute_exact(answer, predicted_answer)
        f1_score = compute_f1(answer, predicted_answer)
        rouge2_f1 = compute_rouge2(answer, predicted_answer)

        if verbose:
            print(f"Q: {question}")
            print(f"Gold Answer: {answer}")
            print(f"Predicted Answer: {answer}")
            print(f"Exact Match: {exact_match}, F1 Score: {f1_score}")
            print(f"ROUGE-2 F1 Score: {rouge2_f1}")
            print("-"*40)

        results.append({
            "question": question,
            "answer": answer,
            "predicted_answer": predicted_answer,
            "context": context if context else None,
            "exact_match": exact_match,
            "f1_score": f1_score,
            "rouge2_f1": rouge2_f1
        })
    return results

In [ ]:
vanilla_evaluation_results = evaluate_qa(vanilla_qa, evaluation_benchmark['qas'])

The function returns a list of evaluation results, one dictionary for each qa item.

In [ ]:
vanilla_evaluation_results[0]

Finally, the `present_results` function aggregates the results for the various qa items and prints the overall result. 

In [ ]:
def present_results(eval_results, exp_name=""):
    print(f"{exp_name} Evaluation Results:")
    exact_matches = [res['exact_match'] for res in eval_results]
    f1_scores = [res['f1_score'] for res in eval_results]
    rouge2_f1 = [res['rouge2_f1'] for res in eval_results]
    print(f"Exact Match: {sum(exact_matches) / len(exact_matches) * 100:.2f}%")
    print(f"F1 Score: {sum(f1_scores) / len(f1_scores) * 100:.2f}%")
    print(f"ROUGE2 F1: {sum(rouge2_f1) / len(rouge2_f1) * 100:.2f}%")

    # print out some evaluation results
    for res in eval_results[:5]:
        print(f"Question: {res['question']}")
        print(f"Gold Answer: {res['answer']}")
        print(f"Predicted Answer: {res['predicted_answer']}")
        print(f"Exact Match: {res['exact_match']}, F1 Score: {res['f1_score']}")
        print("ROUGE-2 F1-score:", res['rouge2_f1'])
        print("-"*40)

In [ ]:
present_results(vanilla_evaluation_results, "Vanilla QA")

**TODO:** Experiment with the prompt template and try to achieve an Exact Match score of at least 5%. You may want to try including an example in the prompt (single-shot prompting).

## Part 3 - Oracle Question Answering

We will now establish an upper bound for a retrieval augmented QA system by providing the correct ("gold") context for each question as part of the prompt. These contexts are available as part of each qa_item in the evaluation_benchmark['qas'] dictionary. 

**TODO**: Write a function `oracle_qa(qa_item)` that takes in a qa_item, inserts both the question **and** the gold context into a prompt template, then passes the prompt to the LLM and returns the answer. The function should behave like the `vanilla_qa` function above, so that we can evaluate it using the same evaluation steps. 

In [ ]:
def oracle_qa(qa_item): # Write this function
    pass

**TODO**: run the `evaluate_qa` function on your `oracle_qa` function and display the results. You should see Exact Match scores above 50% (if not, tinker with the prompt template). 

In [ ]:
oracle_evaluation_results = evaluate_qa(oracle_qa, evaluation_benchmark['qas'])
present_results(oracle_evaluation_results)

## Part 4 - Retrieval-Augmented Question Answering - Word Overlap

Next, we will experiment with various approaches for retrieving relevant contexts from the set of available contexts. We first get the list of all 19035 available candidate contexts from the evaluation_benchmark.

In [ ]:
candidate_contexts = evaluation_benchmark["contexts"]

In [ ]:
len(candidate_contexts)

In [ ]:
candidate_contexts[0]

### Token Overlap Retriever 
Let's first experiment with a simple retriever based on word overlap. Given a question, we measure how many of its tokens appear in each of the candidate contexts. We then retrieve the k contexts with the highest overlap. 

**TODO:** Write the function `retrieve_overlap(question, contexts, top_k)` that takes in the question (a string) and a list of contexts (each context is a string). It should calculate the word overlap between the question and *each* context, and return a list of the *top_k* contexts with the highest overlap. 

In [ ]:
# word overlap retriever -- write this function
def retrieve_overlap(question, contexts, top_k=5):
    return []

The following function runs the retriever a list of qa_items. For each qa_item it obtains the list of retrieved contexts and adds them to the qa_item. 

In [ ]:
def add_rag_context(qa_items, contexts, retriever, top_k=5):
    result_items = copy.deepcopy(qa_items)
    for inst in tqdm.tqdm(result_items, desc="Retrieving contexts"):
        question = inst['question']
        retrieved_contexts = retriever(question, contexts, top_k)
        inst['rag_contexts'] = retrieved_contexts   
    return result_items

In [ ]:
rag_qa_pairs = add_rag_context(evaluation_benchmark['qas'], candidate_contexts, retrieve_overlap)

It returns a copy of the qa_item list that is now annotated with the additional 'rag_contexts'.

In [ ]:
rag_qa_pairs[0]

Before we run an end-to-end evaluation, we can check the accuracy of the word overlap retriever. In other words, for what fraction of questions is the gold context included in the top-k retrieved contexts. 

In [ ]:
# evaluation metric of retriever
def evaluate_retriever(rag_qa_pairs):
    """
    Evaluates the retriever by computing the accuracy of retrieved contexts against reference contexts.
    """
    correct_retrievals = 0
    for qa_item in rag_qa_pairs:
        if qa_item['context'] in qa_item['rag_contexts']:
            correct_retrievals += 1
    accuracy = correct_retrievals / len(rag_qa_pairs)
    return accuracy

In our implementation, we got an accuracy of 0.372. 

In [ ]:
evaluate_retriever(rag_qa_pairs)

**TODO**: Write a function `rag_qa(qa_item)` that behaves like the `vanilla_qa` and `oracle_qa` functions above. Create a prompt from the question and the top-k retrieved contexts (instead of the gold context you used in `oracle_qa`). You can assume that `qa_item` already 
contains the 'rag_contexts' field. 

In [ ]:
def rag_qa(qa_item): # Write this function
    pass 

**TODO**: Like you did for the vanilla and oracle qa system, evaluate the `rag_qa` function and display the results. In our implementation, we got an exact match of 19.6%. 

In [ ]:
rag_overlap_eval = evaluate_qa(rag_qa, rag_qa_pairs)
present_results(rag_overlap_eval)

## Part 5 - Retrieval-Augmented Question Answering - Dense Retrieval

In this step, we will try to will encode each context and questions using BERT. We will then retrieve the k contexts whose embeddings have the highest cosine similarity to the question embedding.

### 5.1 Creating Embeddings for Contexts and Questions 

Here is an example for how to use BERT to encode a sentence. Instead of using the CLS embeddings (as discussed in class) we will pool together the token representations at the last layer by averaging. The resulting representation is a (1,768) tensor. 

In [ ]:
device = "cuda" 
from transformers import BertTokenizer, BertModel # If you run into memory issues, you 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

inputs = tokenizer("This is a sample sentence.", return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
with torch.no_grad(): 
    outputs = model(**inputs)
    hidden_states = outputs.last_hidden_state
    embedding = torch.mean(hidden_states, dim=1)  # (batch_size=1, embedding size =768)

In [ ]:
embedding.shape

**TODO**: Write code to encode each candidate context. Stack the embeddings together into a single (19035, 768) pytorch tensor that we can save to disk and reload as needed (see above for how to access the candidate contexts). On some lower-resource systems you may have trouble instantiating both BERT and OLMo2 at the same time. Storing the encoded representations allows you to run just OLMo for the QA part.

In [ ]:
embedding_list = []

with torch.no_grad(): 
    # context_embeddings = ... 
    pass # Complete this code

    

In [ ]:
torch.save(context_embeddings, "context_embeddings.pt")

**TODO**: Similarly encode each question and stack the embeddings together into a single (250, 768) pytorch tensor that we can save to disk and reload as needed.

In [ ]:
embedding_list = []

with torch.no_grad(): 
    # question_embeddings = ... 
    pass # Complete this code

    

In [ ]:
torch.save(question_embeddings, "question_embeddings.pt")

### 5.2 Similarity Retriever

In [ ]:
context_embeddings = torch.load("context_embeddings.pt")
question_embeddings = torch.load("question_embeddings.pt")

**TODO**: Write a function `retrieve_cosine(question_embedding, contexts, context_embeddings)` that takes in the embedding for a single question (a [1,768] tensor), a list of contexts (each is a string), and the context embedding tensor [19035,768].
Note that the indices of the context list and the rows of the context_embeddings tensor line up. i.e. `context_embeddings[0]` is the embedding for `contexts[0]`, etc.
You can use `torch.nn.functional.cosine_similarity` (or `F.cosine_similarity` since we imported `torch.nn.functional` as `F`, which is conventional) to calculate the similarities efficiently. You may also ant to look at `torch.topk`, but other solutions are possible. 

In [ ]:
def retrieve_cosine(question_emb, contexts, context_embeddings, top_k=5):
    pass # Complete this function 

In [ ]:
retrieve_cosine(question_embeddings[0], candidate_contexts, context_embeddings)

**TODO**: Write a new version of the add_rag_context function we provided above. This function should now additionally take the question embeddings and context embeddings as parameters, run the retrieval for each question (using the retrieve_cosine function above) and populate a new list of qa_items, include the selected 'rag_contexts'.

In [ ]:
def add_rag_context(qa_items, contexts, retriever, question_embeddings, context_embeddings, top_k=5):
    pass # Complete this function

In [ ]:
rag_qa_items = add_rag_context(evaluation_benchmark['qas'], candidate_contexts, retrieve_cosine, question_embeddings, context_embeddings)

In [ ]:
rag_qa_items[0]

Run the `evaluate_retriever` function on the new qa_items. In our experiments, we got an accuracy of about 0.4.

In [ ]:
evaluate_retriever(rag_qa_items)

Then, evaluate the rag_qa approach using the revised rag_qa_items. You should get an Exact match better than 20%.  

In [ ]:
result = evaluate_qa(rag_qa, rag_qa_items)
present_results(result)

## Part 6 - Experiments

**TODO** For the overlap and dense retrievers (from part 5 and 6), what happens when you change the number of retrieved contexts? Present a table of results for k=1, k=5 (already done), k=10, and k=20. 


## Part 7 -Improving the QA System 

**TODO**
In this part, we ask you to come up with one interesting or novel idea for improving the QA system. Your system does *not* have to outperform the models from part 4 or 5, but for full credit you should implement at least one new idea, beyond just changing parameters. You can either work on better retrieval or better QA/LLM performance. Show the full code for the necessary steps and evaluation results. 

Ideas for improving the retriever include: improved word overlap (better tokenization/ text normalization, using TF-IDF, ...), or choosing a different approach or different model (other than BERT) for calculating context and question embeddings.

For the LLM, you could try a different transformer model, including text-to-text models (e.g. T5).                                                                                                           
